<a href="https://colab.research.google.com/github/thedatadj/natural-language-processing/blob/main/autocorrect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this project I created a function called `topsugg` which takes as an argument a misspelled word and a number `n` of number of suggestions, and outputs the `n` most likely suggestions for that misspelled word.

# Dataset
Load and preprocess the dataset.


In [100]:
# Download file
!gdown 14r2LAGtFdedAw4oFsDn9XJ1QBtYb8Ifg

# Load file
file0 = open("/content/shakespeare.txt", "r")

# Read file
file1 = file0.read()

# Make all content lowercase
file2 = file1.lower()

# Tokenize file
import re
words = re.findall(r"\w+", file2)

# Get vocabulary
vocab = set(words)

Downloading...
From: https://drive.google.com/uc?id=14r2LAGtFdedAw4oFsDn9XJ1QBtYb8Ifg
To: /content/shakespeare.txt
100% 307k/307k [00:00<00:00, 118MB/s]


Get a frequency dictionary of the words.

In [101]:
freq = {}
for word in words:
    if word not in freq:
        freq[word] = 0
    freq[word] += 1

# Word probability
Get the probability of each word in `freq`.

In [102]:
prob = {}
totalwords = len(words)
for word in words:
    prob[word] = freq[word]/totalwords

# Edit operations
Create a function for each edit operation (delete, swith, replace, insert) that returns all possibilities for a word.


## Delete
Function which domain are all words from the vocabulary and the range are all possible strings after performing the operation delete.

In [103]:
def odelete(word):
    splits = [(word[:i], word[i:]) for i in range(len(word)+1)]
    deletes = [L + R[1:] for L, R in splits if R]
    return deletes

In [104]:
odelete('cans')

['ans', 'cns', 'cas', 'can']

## Switch
Function that takes in a word, and returns a list containing all possible strings generated after applying a switch between each consecutive letters.

In [105]:
def oswap(word):
    swaps = []
    for i in range(len(word)-1):
        left = word[i:i + 1]
        right = word[i+1:i+2]
        sub0 = word[:i]
        sub1 = word[i+2:]
        newword = sub0 + right + left + sub1
        swaps.append(newword)
    return swaps

In [106]:
oswap('nice')

['ince', 'ncie', 'niec']

## Replace
Function that takes in a word and returns all possible strings generated after applying the replace operation.

In [107]:
def oreplace(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    replace = set()
    for i, c in enumerate(word):
        for l in letters:
            sub0 = word[:i]
            sub1 = word[i + 1:]
            newword = sub0 + l + sub1
            replace.add(newword)

    replace = sorted(list(replace))
    if word in replace:
        replace.remove(word)
    return replace

In [108]:
print(f"Number of outputs of oreplace('at') is {len(oreplace('at'))}")

Number of outputs of oreplace('at') is 50


## Insert
Function that takes in a word and outputs a list of all possible strings after applying the insert operation.

In [109]:
def oinsert(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    inserts = []
    for i in range(len(word)+1):
        for l in letters:
            sub0 = word[:i]
            sub1 = word[i:]
            newword = sub0 + l + sub1
            inserts.append(newword)
    return inserts

In [110]:
len(oinsert('at'))

78

# Edit distance 1
Function which input is a string and outputs all the possible single edits on that string.

In [111]:
def edit1(word, switch=True):
    result = odelete(word) + oinsert(word) + oreplace(word)
    if switch:
        result = result + oswap(word)
    return set(result)

In [112]:
len(edit1('at'))

129

# Edit distance 2
Function that returns all possible transformations of a string after applying 2 edit operations.

In [113]:
def edit2(word, switch=True):
    firstedit = edit1(word)
    secondedit = set()
    for firststrings in firstedit:
        for secondstrings in edit1(firststrings):
            secondedit.add(secondstrings)
    return set(secondedit)

In [114]:
len(edit2('a'))

2654

# Suggestions
Get the most probable word given a misspelled word.

In [181]:
word = 'nise' # Meant nice

def topsugg(word, n=2):
    # Suggestions
    suggestions = []

    if word in vocab:
        suggestions.append(word)
    elif edit1(word).intersection(vocab):
        for string in edit1(word).intersection(vocab):
            suggestions.append(string)
    elif edit2(word).intersection(vocab):
        for string in edit2(word).intersection(vocab):
            suggestions.append(string)
    else:
        suggestions.append(word)

    topdic = {}
    for string in suggestions:
        if string in vocab:
            topdic[string] = prob[string]
        else:
            topdic[string] = 0

    bestword = list(topdic.keys())[0]
    for string in topdic:
        if topdic[bestword] < topdic[string]:
            bestword = string

    # Top n words
    import numpy as np
    dictest = topdic
    values = list(dictest.values())
    valuesa = np.array(values)
    valuesi = np.argsort(valuesa)
    keys = np.array(list(dictest.keys()))
    n_best = list(zip(keys[valuesi[-n:]], valuesa[valuesi[-n:]]))
    return n_best

# Demostration
Given a misspelled word, the function `topsugg` will output the `n` most likely suggested words.

In [183]:
topsugg("dys", 1)

[('days', 0.0004103405826836274)]

<table>
    <tr>
        <td>
            Based on:
        </td>
        <td>
            Assignment from the Natural Language Processing Specialization in Coursera.
        </td>
    </tr>
</table>